In [2]:
import requests
import gzip
import shutil
import os

# download anthology file
url = "https://aclanthology.org/anthology+abstracts.bib.gz"
r = requests.get(url)
open('anthology+abstracts.bib.gz', 'wb').write(r.content)

# open and unpack gz
with gzip.open('anthology+abstracts.bib.gz', 'rb') as f_in:
    with open('anthology+abstracts.bib', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        
os.remove("anthology+abstracts.bib.gz")

In [3]:
"""
from tempfile import mkstemp
from shutil import move, copymode
from os import fdopen, remove


filepath = 'anthology+abstracts.bib'

fd, abspath = mkstemp()
with fdopen(fd,'w') as repl:
    with open(filepath,'r') as bibtex_file:
        for line in bibtex_file:
            if "month = " in line:
                old_cont = line[12:line.find(',')]
                month = line[12:15]
                new_cont = '"{}"'.format(month)
                #line_quot = 'month = "{}"'.format(line[12:15])
                repl.write(line.replace(old_cont, new_cont))
            else:
                repl.write(line)
            
copymode(filepath, abspath)
remove(filepath)
move(abspath, filepath)
"""

'anthology+abstracts.bib'

In [4]:
# watch out: computes very long
import bibtexparser

# parse bib file, output: list of dicts
with open('anthology+abstracts.bib') as bibtex_file:   
    bib_database = bibtexparser.bparser.BibTexParser(common_strings=True).parse_file(bibtex_file)


In [6]:
import pandas as pd

# define keywords & words to exclude
keywords = ['fair', 'fairness', 'race', 'gender', 'bias', 'biases', 'protected attribute', 'protected categor']
excludes = ['hate']
df = pd.DataFrame()

for paper in bib_database.entries:
    if 'abstract' in paper.keys():
        if any(keyword in paper['abstract'] for keyword in keywords):
            if not any(exclude in paper['abstract'] for exclude in excludes):
                temp_row = {}

                # select only select papers from 2016 on
                if 'year' in paper.keys() and int(paper['year']) >= 2016:

                    for cat in paper.keys():
                        temp_row[cat] = paper[cat]

                    df = df.append(temp_row, ignore_index=True)
        
df.to_csv('bias_paper.csv')